In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('marketing_campaign.csv', header=0, sep=';')

print(list(data.columns))


['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response']


In [2]:
#Spending variable creation
data['Age'] = 2021-data['Year_Birth']

data['Spending'] = data['MntWines']+data['MntFruits']+data['MntMeatProducts'] + \
    data['MntFishProducts']+data['MntSweetProducts']+data['MntGoldProds']
#Seniority variable creation
last_date = date(2021, 10, 4)
data['Seniority'] = pd.to_datetime(
    data['Dt_Customer'], dayfirst=True, format='%Y-%m-%d')
data['Seniority'] = pd.to_numeric(data['Seniority'].dt.date.apply(
    lambda x: (last_date - x)).dt.days, downcast='integer')/30
data = data.rename(columns={'NumWebPurchases': "Web",
                   'NumCatalogPurchases': 'Catalog', 'NumStorePurchases': 'Store'})
data['Marital_Status'] = data['Marital_Status'].replace(
    {'Divorced': 'Alone', 'Single': 'Alone', 'Married': 'In couple', 'Together': 'In couple', 'Absurd': 'Alone', 'Widow': 'Alone', 'YOLO': 'Alone'})
data['Education'] = data['Education'].replace(
    {'Basic': 'Undergraduate', '2n Cycle': 'Undergraduate', 'Graduation': 'Postgraduate', 'Master': 'Postgraduate', 'PhD': 'Postgraduate'})

data['Children'] = data['Kidhome']+data['Teenhome']
data['Has_child'] = np.where(data.Children > 0, 'Has child', 'No child')
data['Children'].replace(
    {3: "3 children", 2: '2 children', 1: '1 child', 0: "No child"}, inplace=True)
data = data.rename(columns={'MntWines': "Wines", 'MntFruits': 'Fruits', 'MntMeatProducts': 'Meat',
                   'MntFishProducts': 'Fish', 'MntSweetProducts': 'Sweets', 'MntGoldProds': 'Gold'})


data = data[['Age', 'Education', 'Marital_Status', 'Income', 'Spending', 'Seniority',
             'Has_child', 'Children', 'Wines', 'Fruits', 'Meat', 'Fish', 'Sweets', 'Gold']]
data.head()


,Age,Education,Marital_Status,Income,Spending,Seniority,Has_child,Children,Wines,Fruits,Meat,Fish,Sweets,Gold
0,64,Postgraduate,Alone,58138.0,1617,110.566667,No child,No child,635,88,546,172,88,88
1,67,Postgraduate,Alone,46344.0,27,92.233333,Has child,2 children,11,1,6,2,1,6
2,56,Postgraduate,In couple,71613.0,776,98.866667,No child,No child,426,49,127,111,21,42
3,37,Postgraduate,In couple,26646.0,53,93.100000,Has child,1 child,11,4,20,10,3,5
4,40,Postgraduate,In couple,58293.0,422,93.833333,Has child,1 child,173,43,118,46,27,15


In [3]:
data = data.dropna(subset=['Income'])
data = data[data['Income'] < 600000]


In [4]:
scaler = StandardScaler()
dataset_temp = data[['Income', 'Seniority', 'Spending']]
X_std = scaler.fit_transform(dataset_temp)
X = normalize(X_std, norm='l2')

gmm = GaussianMixture(n_components=4, covariance_type='spherical',
                      max_iter=2000, random_state=5).fit(X)
labels = gmm.predict(X)
dataset_temp['Cluster'] = labels
dataset_temp = dataset_temp.replace(
    {0: 'Stars', 1: 'Need attention', 2: 'High potential', 3: 'Leaky bucket'})
data = data.merge(dataset_temp.Cluster, left_index=True, right_index=True)

pd.options.display.float_format = "{:.0f}".format
summary = data[['Income', 'Spending', 'Seniority', 'Cluster']]
summary.set_index("Cluster", inplace=True)
summary = summary.groupby('Cluster').describe().transpose()
summary.head()


Cluster       High potential  Leaky bucket  Need attention  Stars
Income count             584           641             528    462
       mean            34757         37705           69542  73438
       std             12075         12397           12006  13753
       min              2447          1730           44802  49090
       25%             26489         28839           60880  65298

In [5]:
PLOT = go.Figure()
for C in list(data.Cluster.unique()):

    PLOT.add_trace(go.Scatter3d(x=data[data.Cluster == C]['Income'],
                                y=data[data.Cluster == C]['Seniority'],
                                z=data[data.Cluster == C]['Spending'],
                                mode='markers', marker_size=6, marker_line_width=1,
                                name=str(C)))
PLOT.update_traces(
    hovertemplate='Income: %{x} <br>Seniority: %{y} <br>Spending: %{z}')


PLOT.update_layout(width=800, height=800, autosize=True, showlegend=True,
                   scene=dict(xaxis=dict(title='Income', titlefont_color='black'),
                              yaxis=dict(title='Seniority',
                                         titlefont_color='black'),
                              zaxis=dict(title='Spending', titlefont_color='black')),
                   font=dict(family="Gilroy", color='black', size=12))


In [6]:
#Create Age segment
cut_labels_Age = ['Young', 'Adult', 'Mature', 'Senior']
cut_bins = [0, 30, 45, 65, 120]
data['Age_group'] = pd.cut(data['Age'], bins=cut_bins, labels=cut_labels_Age)
#Create Income segment
cut_labels_Income = ['Low income', 'Low to medium income',
                     'Medium to high income', 'High income']
data['Income_group'] = pd.qcut(data['Income'], q=4, labels=cut_labels_Income)
#Create Seniority segment
cut_labels_Seniority = [
    'New customers', 'Discovering customers', 'Experienced customers', 'Old customers']
data['Seniority_group'] = pd.qcut(
    data['Seniority'], q=4, labels=cut_labels_Seniority)
data = data.drop(columns=['Age', 'Income', 'Seniority'])


In [7]:
cut_labels = ['Low consumer', 'Frequent consumer', 'Biggest consumer']
data['Wines_segment'] = pd.qcut(data['Wines'][data['Wines'] > 0], q=[
                                0, .25, .75, 1], labels=cut_labels).astype("object")
data['Fruits_segment'] = pd.qcut(data['Fruits'][data['Fruits'] > 0], q=[
                                 0, .25, .75, 1], labels=cut_labels).astype("object")
data['Meat_segment'] = pd.qcut(data['Meat'][data['Meat'] > 0], q=[
                               0, .25, .75, 1], labels=cut_labels).astype("object")
data['Fish_segment'] = pd.qcut(data['Fish'][data['Fish'] > 0], q=[
                               0, .25, .75, 1], labels=cut_labels).astype("object")
data['Sweets_segment'] = pd.qcut(data['Sweets'][data['Sweets'] > 0], q=[
                                 0, .25, .75, 1], labels=cut_labels).astype("object")
data['Gold_segment'] = pd.qcut(data['Gold'][data['Gold'] > 0], q=[
                               0, .25, .75, 1], labels=cut_labels).astype("object")
data.replace(np.nan, "Non consumer", inplace=True)
data.drop(columns=['Spending', 'Wines', 'Fruits', 'Meat',
          'Fish', 'Sweets', 'Gold'], inplace=True)
data = data.astype(object)


In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 999)
pd.options.display.float_format = "{:.3f}".format
association = data.copy()
df = pd.get_dummies(association)
min_support = 0.08
max_len = 10
frequent_items = apriori(df, use_colnames=True,
                         min_support=min_support, max_len=max_len + 1)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)

product = 'Wines'
segment = 'Biggest consumer'
target = '{\'%s_segment_%s\'}' % (product, segment)
results_personnal_care = rules[rules['consequents'].astype(str).str.contains(
    target, na=False)].sort_values(by='confidence', ascending=False)
results_personnal_care.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9357,"(Age_group_Mature, Education_Postgraduate, Cluster_Need attention)",(Wines_segment_Biggest consumer),0.121,0.249,0.084,0.698,2.805,0.054,2.486
4860,"(Income_group_High income, Cluster_Need attention)",(Wines_segment_Biggest consumer),0.121,0.249,0.084,0.697,2.800,0.054,2.476
4854,"(Age_group_Mature, Cluster_Need attention)",(Wines_segment_Biggest consumer),0.130,0.249,0.090,0.692,2.782,0.058,2.439
4872,"(Seniority_group_Old customers, Cluster_Need attention)",(Wines_segment_Biggest consumer),0.123,0.249,0.085,0.691,2.779,0.054,2.433
9385,"(Education_Postgraduate, Cluster_Need attention, Meat_segment_Biggest consumer)",(Wines_segment_Biggest consumer),0.122,0.249,0.081,0.664,2.670,0.051,2.237
